In [2]:
import os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time

In [3]:
base_url = 'https://www.amazon.com/stores/page/E0960A73-FD40-4EBA-8152-512746435C73'

In [4]:
chromedriver = "C:\SeleniumDrivers\chromedriver"
driver = webdriver.Chrome(chromedriver)
#driver.maximize_window()
driver.implicitly_wait(45)

In [5]:
driver.get(base_url)
time.sleep(30)

### codes for testing

product_info = driver.find_elements_by_class_name('ProductGridItem__fixed__1w9d4')


product_info

ppp = product_info[0]

pp1 = ppp.find_elements_by_tag_name('a')[2]
pp1

pp1.get_property('href')

product_link = []
for elem in product_info:
    p_tag = elem.find_elements_by_tag_name('a')[2]
    p_link = p_tag.get_property('href')
    product_link.append(p_link)
print(product_link)

len(product_link)

button = driver.find_elements_by_css_selector('.style__button__1KJD9')
button[0].click()

In [6]:
product_links =[]
condition = True
while condition:
    product_info = driver.find_elements_by_class_name('ProductGridItem__fixed__1w9d4')
    for elem in product_info:
        p_tag = elem.find_elements_by_tag_name('a')[2]
        p_link = p_tag.get_property('href')
        if p_link not in product_links:
            product_links.append(p_link)
    try:
        button = driver.find_elements_by_css_selector('.style__button__1KJD9')
        button[0].click()
        time.sleep(2)
    except:
        condition = False
        

In [7]:
len(product_links)

67

In [26]:
driver.get(product_links[0])

In [74]:
title = driver.find_element_by_xpath('//*[@id="productTitle"]').text# title
ASIN = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[1]/td').text           # ASIN
Price = driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text                                           # price
Review = driver.find_element_by_xpath('//*[@id="a-popover-content-1"]/div/div/div/div[1]/span')                                        # review
Ranking = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[3]/td').text.split('\n')[0]          # best_seler
avv = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/th').text

driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[3]/th').text

'Best Sellers Rank'

In [73]:
avv

'Customer Reviews'

In [71]:
Review.det

<selenium.webdriver.remote.webelement.WebElement (session="fd1381bd41cc0e567b64512c9a5fed61", element="07c0baec-73eb-4afa-b54b-0d9fa7e7e786")>

In [65]:
data1 = {
    'Title': title,
    'ASIN': ASIN,
    'Price': Price,
    'Review':Review,
    'Ranking': Ranking
}

In [66]:
print(data1)

{'Title': 'HP 15z 15-ef0021n Home and Business Laptop (AMD Ryzen 3 3250U 2-Core, 16GB RAM, 512GB PCIe SSD, 15.6" HD (1366x768), AMD Radeon Graphics, WiFi, Bluetooth, Webcam, 2xUSB 3.1, Win 10 Home) with USB Hub', 'ASIN': 'B08KQ3KTQ2', 'Price': '$565.00', 'Review': '', 'Ranking': '#26,900 in Computers & Accessories (See Top 100 in Computers & Accessories)'}


In [9]:
import pandas as pd

In [10]:
from tqdm import tqdm
df = pd.DataFrame(columns = ['Title', 'ASIN', 'Price', 'Review','Ranking'])
for link in tqdm(product_links):
    driver.get(link)
    title = driver.find_element_by_xpath('//*[@id="productTitle"]').text                                                    
    ASIN = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[1]/td').text         
    try:
        Price = driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text
    except:
        Price = ''
    try:
        if driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/th').text == 'Customer Reviews':
            Review = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/td').text.split('\n')
    except:
        Review = ''
    try: 
        if driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[3]/th').text == 'Best Sellers Rank':
            Ranking = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[3]/td').text.split('\n')
        elif driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/th').text == 'Best Sellers Rank':
            Ranking = driver.find_element_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/td').text.split('\n')
    except:
        Ranking = ''
    product_details = {
        'Title': title,
        'ASIN': ASIN,
        'Price': Price,
        'Review':Review,
        'Ranking': Ranking
    }
    df = df.append(product_details, ignore_index = True)

    print(df.tail(1))
    
df.to_excel('products.xlsx')

  1%|█▏                                                                                 | 1/67 [00:32<36:17, 32.99s/it]

                                               Title        ASIN    Price  \
0  HP 15z 15-ef0021n Home and Business Laptop (AM...  B08KQ3KTQ2  $565.00   

                             Review  \
0  [10 ratings, 5.0 out of 5 stars]   

                                             Ranking  
0  [#28,199 in Computers & Accessories (See Top 1...  


  3%|██▍                                                                                | 2/67 [00:41<20:17, 18.73s/it]

                                               Title        ASIN    Price  \
1  HP 17z-cp000 Home & Business Laptop (AMD Athlo...  B098LG3GWB  $549.00   

                             Review  \
1  [10 ratings, 5.0 out of 5 stars]   

                                             Ranking  
1  [#103,569 in Computers & Accessories (See Top ...  


  4%|███▋                                                                               | 3/67 [00:48<14:08, 13.26s/it]

...

                                                Title        ASIN      Price  \
64  ASUS ROG Zephyrus Duo SE 15 Gaming and Enterta...  B094HZYLG8  $4,554.99   

                             Review Ranking  
64  [2 ratings, 3.9 out of 5 stars]          


 99%|████████████████████████████████████████████████████████████████████████████████▊ | 66/67 [49:08<00:38, 38.47s/it]

                                                Title        ASIN      Price  \
65  Lenovo IdeaPad 330 Home and Business Laptop (I...  B07V2BDR4F  $1,338.00   

                             Review  \
65  [2 ratings, 3.9 out of 5 stars]   

                                              Ranking  
65  [#420,306 in Computers & Accessories (See Top ...  


100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [49:14<00:00, 44.10s/it]

                                                Title        ASIN      Price  \
66  Lenovo IdeaPad 330 Home and Business Laptop (I...  B07V4GHG64  $1,132.00   

                             Review  \
66  [2 ratings, 3.9 out of 5 stars]   

                                              Ranking  
66  [#362,366 in Computers & Accessories (See Top ...  


NameError: name 'df' is not defined